In [1]:
!wget "https://doc-0s-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/audtgi65fb7h3uqsf14bcbiigedi4lcs/1525406400000/13182073909007362810/*/0B7EVK8r0v71pZjFTYXZWM3FlRnM?e=download"

--2018-05-04 05:07:47--  https://doc-0s-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/audtgi65fb7h3uqsf14bcbiigedi4lcs/1525406400000/13182073909007362810/*/0B7EVK8r0v71pZjFTYXZWM3FlRnM?e=download
Resolving doc-0s-84-docs.googleusercontent.com (doc-0s-84-docs.googleusercontent.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting to doc-0s-84-docs.googleusercontent.com (doc-0s-84-docs.googleusercontent.com)|74.125.141.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘0B7EVK8r0v71pZjFTYXZWM3FlRnM?e=download’

0B7EVK8r0v71pZjFTYX     [            <=>     ]   1.34G   114MB/s    in 11s     

2018-05-04 05:07:59 (125 MB/s) - ‘0B7EVK8r0v71pZjFTYXZWM3FlRnM?e=download’ saved [1443490838]



In [0]:
!mv "0B7EVK8r0v71pZjFTYXZWM3FlRnM?e=download" "celeba.zip"

In [0]:
!mv img_align_celeba data

In [0]:
def get_batch(image_files, width, height, mode):
        data_batch = np.array(
            [get_image(sample_file, width, height, mode) for sample_file in image_files])

        return data_batch

def get_image(image_path, width, height, mode):
    
        image = Image.open(image_path)
        # image = image.resize([width, height], Image.BILINEAR)
        if image.size != (width, height):  
        # Remove most pixels that aren't part of a face
            face_width = face_height = 108
            j = (image.size[0] - face_width) // 2
            i = (image.size[1] - face_height) // 2
            image = image.crop([j, i, j + face_width, i + face_height])
            image = image.resize([width, height])
    
        return np.array(image.convert(mode))

In [0]:
import os
import sys
import os
from PIL import Image
from glob import glob
import numpy as np

In [4]:
from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
from keras.optimizers import Adam
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "<decorator-gen-121>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/applicati

In [0]:
def build_generator(noise=(100,)):
    input = Input(noise)
    layer = Dense(128 * 8 * 8, activation="relu")(input)
    layer = Reshape((8, 8, 128))(layer)
    layer = BatchNormalization(momentum=0.8)(layer) # check the formula
    layer = UpSampling2D()(layer)
    layer = Conv2D(128, kernel_size=3, padding="same")(layer)
    layer = Activation("relu")(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = UpSampling2D()(layer)
    layer = Conv2D(64, kernel_size=3, padding="same")(layer)
    layer = Activation("relu")(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = Conv2D(3, kernel_size=3, padding="same")(layer)
    out = Activation("tanh")(layer) # test karke dekha tha tanh is advised by the author itself
    model = Model(input, out)
    print(" Generator Model Summary ")
    model.summary()
    return model

In [0]:
def build_discriminator(input_img):
    input = Input(input_img)
    layer =Conv2D(32, kernel_size=3, strides=2, padding="same")(input)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Dropout(0.25)(layer)
    layer = Conv2D(64, kernel_size=3, strides=2, padding="same")(layer)
    layer = (LeakyReLU(alpha=0.2))(layer)
    layer = Dropout(0.25)(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = Conv2D(128, kernel_size=3, strides=2, padding="same")(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Dropout(0.25)(layer)
    layer = BatchNormalization(momentum=0.8)(layer)
    layer = Conv2D(256, kernel_size=3, strides=1, padding="same")(layer)
    layer = LeakyReLU(alpha=0.2)(layer)
    layer = Dropout(0.25)(layer)
    layer = Flatten()(layer)
    out = Dense(1, activation='sigmoid')(layer)

    model = Model(input, out)
    print(" Discriminator Model Summary ")
    model.summary()
    return model

In [0]:
def train(generator, discriminator, stacked, epochs=2000, batchNo=128):
    data_dir = './data'
    train_arr = get_batch(glob(os.path.join(data_dir, '*.jpg'))[:10000], 32, 32, 'RGB')
    
    train_arr = (train_arr.astype(np.float32) - 127.5) / 127.5
    
    num_batches = int(train_arr.shape[0] / float(batchNo))
    
    print('Number of examples: ', train_arr.shape[0])
    print('Number of Batches: ', num_batches)
    print('Number of epochs: ', epochs)

    half_batchNo = int(batchNo / 2)
    
    accuracy = []


    for epoch in range(epochs + 1):
        print("Epoch: " + str(epoch))
        for batch in range(num_batches):
            print("Batch: " + str(batch) + "/" + str(num_batches))
    
            # noise images for the batch
            noise = np.random.normal(0, 1, (half_batchNo, 100))
        
            generateImg = generator.predict(noise)
            zero_Label = np.zeros((half_batchNo, 1))
    
            # real images for batch
            idx = np.random.randint(0, train_arr.shape[0], half_batchNo)
            datasetImg = train_arr[idx]
            one_Label = np.ones((half_batchNo, 1))
    
            # Train the discriminator (real classified as ones and generated as zeros)
            disc_TrainR = discriminator.train_on_batch(datasetImg, one_Label)
            disc_TrainF = discriminator.train_on_batch(generateImg, zero_Label)
            mean_Loss = 0.5 * np.add(disc_TrainR, disc_TrainF)
    
            noise = np.random.normal(0, 1, (batchNo, 100))
            # Train the generator
            gen_Train = stacked.train_on_batch(noise, np.ones((batchNo, 1)))
    
            # Plot the progress
            print("Epoch %d Batch %d/%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (epoch, batch, num_batches, mean_Loss[0], 100 * mean_Loss[1], gen_Train))
            
            accuracy.append([ ((epoch+1)*batch) , (100*mean_Loss[1])] )
            
            if batch % 50 == 0:
                save_imgs(generator, epoch, batch)
            
            accuracy_a = np.array(accuracy)
    
    plt.plot(accuracy_a[:,0], accuracy_a[:,1], label="Discriminator ACCURACY ")
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy of Discriminator over epochs for CelebA dataset On Convolutional Model')
    plt.grid(True)
    plt.show()  

In [0]:
def save_imgs(generator, epoch, batch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = (1/2.5) * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, :])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig("images/Celeb_%d_%d.png" % (epoch, batch))
    plt.close()

In [0]:
!mkdir images

In [0]:
def build_models():

   #start karte hai 0.002 k learning rate se 
   # what is beta_1 ????
    Gopt = Adam(lr=0.0002, beta_1=0.5)
    Dopt = Adam(lr=0.0002, beta_1=0.5)

    discriminator = build_discriminator(input_img=(32, 32, 3))
    discriminator.compile(loss='binary_crossentropy',
                               optimizer=Dopt,
                               metrics=['accuracy'])

    generator = build_generator()
    generator.compile(loss='binary_crossentropy', optimizer=Gopt)

    x = Input(shape=(100,))
    img = generator(x)
    discriminator.trainable = False 
    real = discriminator(img)
    stacked = Model(x, real)
    stacked.compile(loss='binary_crossentropy', optimizer=Gopt)
    return generator, discriminator, stacked

In [16]:
if __name__ == '__main__':
    generator, discriminator, stacked = build_models()
    train(generator, discriminator, stacked, epochs=7, batchNo=32)

 Discriminator Model Summary 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 8, 8, 64) 

Epoch 0 Batch 35/312 [D loss: 0.005496, acc.: 100.00%] [G loss: 0.011371]
Batch: 36/312
Epoch 0 Batch 36/312 [D loss: 0.009369, acc.: 100.00%] [G loss: 0.007002]
Batch: 37/312
Epoch 0 Batch 37/312 [D loss: 0.005915, acc.: 100.00%] [G loss: 0.006011]
Batch: 38/312
Epoch 0 Batch 38/312 [D loss: 0.006326, acc.: 100.00%] [G loss: 0.006908]
Batch: 39/312
Epoch 0 Batch 39/312 [D loss: 0.005544, acc.: 100.00%] [G loss: 0.006358]
Batch: 40/312
Epoch 0 Batch 40/312 [D loss: 0.006393, acc.: 100.00%] [G loss: 0.007348]
Batch: 41/312
Epoch 0 Batch 41/312 [D loss: 0.005540, acc.: 100.00%] [G loss: 0.010831]
Batch: 42/312
Epoch 0 Batch 42/312 [D loss: 0.004402, acc.: 100.00%] [G loss: 0.007001]
Batch: 43/312
Epoch 0 Batch 43/312 [D loss: 0.003600, acc.: 100.00%] [G loss: 0.007308]
Batch: 44/312
Epoch 0 Batch 44/312 [D loss: 0.003334, acc.: 100.00%] [G loss: 0.005486]
Batch: 45/312
Epoch 0 Batch 45/312 [D loss: 0.003489, acc.: 100.00%] [G loss: 0.005463]
Batch: 46/312
Epoch 0 Batch 46/312 [D loss: 0.

Epoch 0 Batch 111/312 [D loss: 0.001103, acc.: 100.00%] [G loss: 0.001257]
Batch: 112/312
Epoch 0 Batch 112/312 [D loss: 0.001048, acc.: 100.00%] [G loss: 0.000702]
Batch: 113/312
Epoch 0 Batch 113/312 [D loss: 0.000839, acc.: 100.00%] [G loss: 0.001227]
Batch: 114/312
Epoch 0 Batch 114/312 [D loss: 0.000855, acc.: 100.00%] [G loss: 0.001043]
Batch: 115/312
Epoch 0 Batch 115/312 [D loss: 0.001504, acc.: 100.00%] [G loss: 0.001171]
Batch: 116/312
Epoch 0 Batch 116/312 [D loss: 0.000584, acc.: 100.00%] [G loss: 0.000929]
Batch: 117/312
Epoch 0 Batch 117/312 [D loss: 0.001262, acc.: 100.00%] [G loss: 0.001556]
Batch: 118/312
Epoch 0 Batch 118/312 [D loss: 0.000575, acc.: 100.00%] [G loss: 0.001019]
Batch: 119/312
Epoch 0 Batch 119/312 [D loss: 0.000891, acc.: 100.00%] [G loss: 0.001051]
Batch: 120/312
Epoch 0 Batch 120/312 [D loss: 0.000790, acc.: 100.00%] [G loss: 0.001207]
Batch: 121/312
Epoch 0 Batch 121/312 [D loss: 0.000537, acc.: 100.00%] [G loss: 0.000740]
Batch: 122/312
Epoch 0 Ba

Epoch 0 Batch 188/312 [D loss: 0.000266, acc.: 100.00%] [G loss: 0.000376]
Batch: 189/312
Epoch 0 Batch 189/312 [D loss: 0.000579, acc.: 100.00%] [G loss: 0.000649]
Batch: 190/312
Epoch 0 Batch 190/312 [D loss: 0.000419, acc.: 100.00%] [G loss: 0.000392]
Batch: 191/312
Epoch 0 Batch 191/312 [D loss: 0.000280, acc.: 100.00%] [G loss: 0.000505]
Batch: 192/312
Epoch 0 Batch 192/312 [D loss: 0.000297, acc.: 100.00%] [G loss: 0.000570]
Batch: 193/312
Epoch 0 Batch 193/312 [D loss: 0.000234, acc.: 100.00%] [G loss: 0.000551]
Batch: 194/312
Epoch 0 Batch 194/312 [D loss: 0.000397, acc.: 100.00%] [G loss: 0.000377]
Batch: 195/312
Epoch 0 Batch 195/312 [D loss: 0.000300, acc.: 100.00%] [G loss: 0.000594]
Batch: 196/312
Epoch 0 Batch 196/312 [D loss: 0.000263, acc.: 100.00%] [G loss: 0.000501]
Batch: 197/312
Epoch 0 Batch 197/312 [D loss: 0.000274, acc.: 100.00%] [G loss: 0.000474]
Batch: 198/312
Epoch 0 Batch 198/312 [D loss: 0.000227, acc.: 100.00%] [G loss: 0.000581]
Batch: 199/312
Epoch 0 Ba

Epoch 0 Batch 264/312 [D loss: 0.000273, acc.: 100.00%] [G loss: 0.000414]
Batch: 265/312
Epoch 0 Batch 265/312 [D loss: 0.000266, acc.: 100.00%] [G loss: 0.000372]
Batch: 266/312
Epoch 0 Batch 266/312 [D loss: 0.000138, acc.: 100.00%] [G loss: 0.000266]
Batch: 267/312
Epoch 0 Batch 267/312 [D loss: 0.000213, acc.: 100.00%] [G loss: 0.000159]
Batch: 268/312
Epoch 0 Batch 268/312 [D loss: 0.000115, acc.: 100.00%] [G loss: 0.000296]
Batch: 269/312
Epoch 0 Batch 269/312 [D loss: 0.000126, acc.: 100.00%] [G loss: 0.000230]
Batch: 270/312
Epoch 0 Batch 270/312 [D loss: 0.000235, acc.: 100.00%] [G loss: 0.000310]
Batch: 271/312
Epoch 0 Batch 271/312 [D loss: 0.000171, acc.: 100.00%] [G loss: 0.000171]
Batch: 272/312
Epoch 0 Batch 272/312 [D loss: 0.000274, acc.: 100.00%] [G loss: 0.000175]
Batch: 273/312
Epoch 0 Batch 273/312 [D loss: 0.000120, acc.: 100.00%] [G loss: 0.000195]
Batch: 274/312
Epoch 0 Batch 274/312 [D loss: 0.000148, acc.: 100.00%] [G loss: 0.000192]
Batch: 275/312
Epoch 0 Ba

Epoch 1 Batch 26/312 [D loss: 0.000072, acc.: 100.00%] [G loss: 0.000155]
Batch: 27/312
Epoch 1 Batch 27/312 [D loss: 0.000102, acc.: 100.00%] [G loss: 0.000195]
Batch: 28/312
Epoch 1 Batch 28/312 [D loss: 0.000114, acc.: 100.00%] [G loss: 0.000132]
Batch: 29/312
Epoch 1 Batch 29/312 [D loss: 0.000120, acc.: 100.00%] [G loss: 0.000175]
Batch: 30/312
Epoch 1 Batch 30/312 [D loss: 0.000119, acc.: 100.00%] [G loss: 0.000129]
Batch: 31/312
Epoch 1 Batch 31/312 [D loss: 0.000108, acc.: 100.00%] [G loss: 0.000152]
Batch: 32/312
Epoch 1 Batch 32/312 [D loss: 0.000189, acc.: 100.00%] [G loss: 0.000212]
Batch: 33/312
Epoch 1 Batch 33/312 [D loss: 0.000153, acc.: 100.00%] [G loss: 0.000106]
Batch: 34/312
Epoch 1 Batch 34/312 [D loss: 0.000069, acc.: 100.00%] [G loss: 0.000118]
Batch: 35/312
Epoch 1 Batch 35/312 [D loss: 0.000135, acc.: 100.00%] [G loss: 0.000142]
Batch: 36/312
Epoch 1 Batch 36/312 [D loss: 0.000162, acc.: 100.00%] [G loss: 0.000240]
Batch: 37/312
Epoch 1 Batch 37/312 [D loss: 0.

KeyboardInterrupt: ignored

In [17]:
!ls images

Celeb_0_0.png	 Celeb_1_200.png  Celeb_2_50.png   Celeb_4_150.png
Celeb_0_100.png  Celeb_1_250.png  Celeb_3_0.png    Celeb_4_200.png
Celeb_0_150.png  Celeb_1_300.png  Celeb_3_100.png  Celeb_4_250.png
Celeb_0_200.png  Celeb_1_50.png   Celeb_3_150.png  Celeb_4_300.png
Celeb_0_250.png  Celeb_2_0.png	  Celeb_3_200.png  Celeb_4_50.png
Celeb_0_300.png  Celeb_2_100.png  Celeb_3_250.png  Celeb_5_0.png
Celeb_0_50.png	 Celeb_2_150.png  Celeb_3_300.png  Celeb_5_100.png
Celeb_1_0.png	 Celeb_2_200.png  Celeb_3_50.png   Celeb_5_50.png
Celeb_1_100.png  Celeb_2_250.png  Celeb_4_0.png
Celeb_1_150.png  Celeb_2_300.png  Celeb_4_100.png
